In [2]:
%matplotlib inline

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import random
import string
import copy

In [3]:
import pickle
with open("data.pickle", 'rb') as f:
    x_train = pickle.load(f)
    y_train = pickle.load(f)
    batch_size = pickle.load(f)
    x_validate = pickle.load(f)
    y_validate = pickle.load(f)

In [4]:
im_rows = 64
im_cols = 32
batch_size = batch_size
im_shape = (im_rows, im_cols, 1)

In [5]:
cnn_model = keras.models.load_model('trained_model_with_data.h5')
cnn_copy = keras.models.load_model('trained_model_with_data.h5')


In [ ]:
import tensorflow_model_optimization as tfm
import tempfile

plm = tfm.sparsity.keras.prune_low_magnitude
end_step = np.ceil(10/batch_size).astype(np.int32)*20

pruning_params = {'pruning_schedule': tfm.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.90,
                                                               begin_step=0,
                                                               end_step=end_step)}
cnn_copy = keras.models.load_model('trained_model_with_data.h5')
cnn_model = keras.models.load_model('trained_model_with_data.h5')

model_for_pruning = plm(cnn_model, **pruning_params)

model_for_pruning.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

logdir = tempfile.mkdtemp()

i = 10
x_fit, y_fit = x_validate[:i], y_validate[:i]

callbacks = [tfm.sparsity.keras.UpdatePruningStep(), tfm.sparsity.keras.PruningSummaries(log_dir=logdir)]
model_for_pruning.fit(x_fit, y_fit, batch_size=batch_size, epochs=5, callbacks=callbacks, verbose=0)
s1 = model_for_pruning.evaluate(x_validate, y_validate)
cnn_copy.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])
cnn_copy.fit(x_fit, y_fit, batch_size=batch_size, epochs=10, callbacks=callbacks, verbose=0)
s1 = model_for_pruning.evaluate(x_validate, y_validate)
s2 = cnn_copy.evaluate(x_validate, y_validate)
print(i)
print(s0)
print()
print(s1)
print()
print(s2)